In [83]:
import pandas as pd
import requests
import json
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import statistics

In [84]:
url = 'https://api.apispreadsheets.com/api/dataset/bank-marketing/'

In [85]:
response = requests.get(url)
dataset = json.loads(response.text)

In [86]:
dataset["data"][0].keys()

dict_keys(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'])

In [87]:
dataset

{'data': [{'age': 30,
   'job': 'unemployed',
   'marital': 'married',
   'education': 'primary',
   'default': 'no',
   'balance': 1787,
   'housing': 'no',
   'loan': 'no',
   'contact': 'cellular',
   'day': 19,
   'month': 'oct',
   'duration': 79.0,
   'campaign': 1,
   'pdays': -1.0,
   'previous': 0,
   'poutcome': 'unknown',
   'y': 'no'},
  {'age': 33,
   'job': 'services',
   'marital': 'married',
   'education': 'secondary',
   'default': 'no',
   'balance': 4789,
   'housing': 'yes',
   'loan': 'yes',
   'contact': 'cellular',
   'day': 11,
   'month': 'may',
   'duration': 220.0,
   'campaign': 1,
   'pdays': 339.0,
   'previous': 4,
   'poutcome': 'failure',
   'y': 'no'},
  {'age': 35,
   'job': 'management',
   'marital': 'single',
   'education': 'tertiary',
   'default': 'no',
   'balance': 1350,
   'housing': 'yes',
   'loan': 'no',
   'contact': 'cellular',
   'day': 16,
   'month': 'apr',
   'duration': 185.0,
   'campaign': 1,
   'pdays': 330.0,
   'previous': 1,


In [88]:
df = pd.json_normalize(dataset["data"])

In [89]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79.0,1,-1.0,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220.0,1,339.0,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185.0,1,330.0,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199.0,4,-1.0,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226.0,1,-1.0,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329.0,5,-1.0,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153.0,1,-1.0,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151.0,11,-1.0,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129.0,4,211.0,3,other,no


In [90]:
feature_df = df.drop(columns=['y'])
y = df['y']
feature_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79.0,1,-1.0,0,unknown
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220.0,1,339.0,4,failure
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185.0,1,330.0,1,failure
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199.0,4,-1.0,0,unknown
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226.0,1,-1.0,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329.0,5,-1.0,0,unknown
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153.0,1,-1.0,0,unknown
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151.0,11,-1.0,0,unknown
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129.0,4,211.0,3,other


In [91]:
y

0       no
1       no
2       no
3       no
4       no
        ..
4516    no
4517    no
4518    no
4519    no
4520    no
Name: y, Length: 4521, dtype: object

In [92]:
X = pd.get_dummies(feature_df)
X

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79.0,1,-1.0,0,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,33,4789,11,220.0,1,339.0,4,False,False,False,...,False,False,True,False,False,False,True,False,False,False
2,35,1350,16,185.0,1,330.0,1,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,30,1476,3,199.0,4,-1.0,0,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,59,0,5,226.0,1,-1.0,0,False,True,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,-333,30,329.0,5,-1.0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4517,57,-3313,9,153.0,1,-1.0,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4518,57,295,19,151.0,11,-1.0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4519,28,1137,6,129.0,4,211.0,3,False,True,False,...,False,False,False,False,False,False,False,True,False,False


In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [94]:
for nn_neighbors in range(30):
    if nn_neighbors == 0:
        continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {nn_neighbors} neighbors: {round(accuracy*100,2)}")

Accuracy for 1 neighbors: 83.2
Accuracy for 2 neighbors: 86.19
Accuracy for 3 neighbors: 85.19
Accuracy for 4 neighbors: 85.3
Accuracy for 5 neighbors: 85.41
Accuracy for 6 neighbors: 85.97
Accuracy for 7 neighbors: 85.41
Accuracy for 8 neighbors: 85.86
Accuracy for 9 neighbors: 85.86
Accuracy for 10 neighbors: 86.85
Accuracy for 11 neighbors: 87.07
Accuracy for 12 neighbors: 86.85
Accuracy for 13 neighbors: 87.18
Accuracy for 14 neighbors: 87.07
Accuracy for 15 neighbors: 87.4
Accuracy for 16 neighbors: 87.07
Accuracy for 17 neighbors: 87.18
Accuracy for 18 neighbors: 87.29
Accuracy for 19 neighbors: 87.4
Accuracy for 20 neighbors: 87.4
Accuracy for 21 neighbors: 87.73
Accuracy for 22 neighbors: 87.18
Accuracy for 23 neighbors: 87.4
Accuracy for 24 neighbors: 87.18
Accuracy for 25 neighbors: 87.29
Accuracy for 26 neighbors: 87.51
Accuracy for 27 neighbors: 87.4
Accuracy for 28 neighbors: 87.18
Accuracy for 29 neighbors: 87.07


In [95]:
# K fold cross validation
from sklearn.model_selection import cross_val_score
for nn_neighbors in range(30):
    if nn_neighbors == 0:
        continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    accuracy = cross_val_score(knn, X, y, cv=5)
    print(f"Accuracy for {nn_neighbors} neighbors: {round(statistics.mean(accuracy)*100,2)}")

Accuracy for 1 neighbors: 85.09
Accuracy for 2 neighbors: 87.97
Accuracy for 3 neighbors: 86.68
Accuracy for 4 neighbors: 87.81
Accuracy for 5 neighbors: 87.3
Accuracy for 6 neighbors: 87.59
Accuracy for 7 neighbors: 87.28
Accuracy for 8 neighbors: 87.86
Accuracy for 9 neighbors: 87.79
Accuracy for 10 neighbors: 88.21
Accuracy for 11 neighbors: 88.1
Accuracy for 12 neighbors: 88.3
Accuracy for 13 neighbors: 88.1
Accuracy for 14 neighbors: 88.48
Accuracy for 15 neighbors: 88.43
Accuracy for 16 neighbors: 88.54
Accuracy for 17 neighbors: 88.5
Accuracy for 18 neighbors: 88.65
Accuracy for 19 neighbors: 88.56
Accuracy for 20 neighbors: 88.61
Accuracy for 21 neighbors: 88.72
Accuracy for 22 neighbors: 88.76
Accuracy for 23 neighbors: 88.74
Accuracy for 24 neighbors: 88.7
Accuracy for 25 neighbors: 88.65
Accuracy for 26 neighbors: 88.63
Accuracy for 27 neighbors: 88.68
Accuracy for 28 neighbors: 88.7
Accuracy for 29 neighbors: 88.61


In [96]:
# improvements

In [97]:
# scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled

array([[-0.4851801 , -0.47084351,  0.50135724, ..., -0.20803258,
        -0.16777661,  0.46989016],
       [-0.9573291 , -0.09336797, -1.19294909, ..., -0.20803258,
        -0.16777661,  0.46989016],
       [-0.8628993 , -0.13630831, -0.46681781, ..., -0.20803258,
        -0.16777661,  0.46989016],
       ...,
       [-0.8628993 , -0.16127362,  1.3485104 , ..., -0.20803258,
        -0.16777661,  0.46989016],
       [ 1.78113509,  1.13359396, -1.31397097, ..., -0.20803258,
        -0.16777661,  0.46989016],
       [ 1.59227549,  2.26468909,  1.83259792, ..., -0.20803258,
        -0.16777661,  0.46989016]])

In [98]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [99]:
X_test_scaled = scaler.transform(X_test)
print("accuracy with scaling", knn.score(X_test_scaled, y_test)*100)

accuracy with scaling 86.96132596685084


In [100]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.876243093922652